## Practical Session 6 - scraping the web with urllib3 and beautifulsoup

Students (pair):
- [Félix Monnier]([link](https://github.com/felixmnnr))
- [Henri Vasseur]([link](https://github.com/Henrivasseur))

**Useful references for this lab**:

[1] `urllib3`: [documentation](https://urllib3.readthedocs.io/en/latest/)

[2] `beautifulsoup4`: [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) 


## <a name="content">Contents</a>
- [Exercise 1: Parsing the content of a web page](#ex1)
- [Exercise 2: Extracting information from Wikipedia](#ex2)
---

 This notebook is aimed at introducing Python functions and library to automatically collect data from static web pages. In particular, this session will be devoted to the `urllib3` and `Beautiful Soup` packages.

 Other useful packages in this context:
 - `os` & `sys` to issue system instructions;
 - `re` for [**r**egular **e**xpressions when manipulating text strings](https://docs.python.org/3/library/re.html). The test the validity of a regular expression;
 - `datetime` to interact with dates & times.

In [1]:
import os
import re
import sys
from datetime import date, datetime

import urllib3
from bs4 import BeautifulSoup

%load_ext autoreload
%autoreload 2

 To take Centrale Lille's proxy into account:

In [2]:
# If proxy : to get out through Centrale Lille's proxy
centrale_proxy = False
if centrale_proxy:
    proxy = urllib3.ProxyManager("http://cache.ec-lille.fr:3128")
else:
    proxy = urllib3.PoolManager()

# See https://stackoverflow.com/questions/40490187/get-proxy-address-on-auto-proxy-discovery-mode-on-mac-os-x
# scutil --proxy

## <a name="ex1">Exercise/example 1: parsing the content of a web page</a> [(&#8593;)](#content)

This example consits in retrieving the version number of the Beautiful Soup package, appearing in the top left corner of the associated [documentation webpage](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). To do this, you can for isntance use the following instructions.

In [3]:
import certifi

cert_path = certifi.where()

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=cert_path)
response = http.request("GET", "https://www.crummy.com/software/BeautifulSoup/bs4/doc/")

 Transform content into formatted text:

In [4]:
utf8_text = response.data.decode("utf-8")
#print(soup)

 Look for the version number and print it:

In [5]:
# Search data using Regex (regular expressions)
# Test regex http://regexr.com
regex = "Beautiful Soup (\d\.)(\d\d\.)(\d)"  # looking for version number under the form Beautiful Soup 4.9.0
web_text = re.search(regex, utf8_text)
print(web_text.group(0))

Beautiful Soup 4.12.0


1\. Extract only the version number from the same page.

> Hint: two useful pages about regular expressions (regexp): [tutorial](https://www.lucaswillems.com/fr/articles/25/tutoriel-pour-maitriser-les-expressions-regulieres), [verifying validity of an expression](http://regexr.com).

Your answers(s)

In [6]:
regex = "(\d\.)(\d\d\.)(\d)"
web_text = re.search(regex, utf8_text)
print(web_text.group(0))

4.12.0


2\. Take a look at the quickstart page of [`Beautiful Soup` (bs4 package)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), and use this library to retrieve the same information.

> Hint:
> - [this page on Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree) can be useful
> - useful elements of code:
>
>```python
> from bs4 import BeautifulSoup
> html_doc = proxy.request('GET','https://www.crummy.com/software/BeautifulSoup/bs4/doc/')
> soup = BeautifulSoup(html_doc, 'html.parser')
> ...
>```

Your answers(s)

In [7]:
if response.status == 200:
    soup = BeautifulSoup(response.data, 'html.parser')
else:
    print(f"Erreur de demande: {response.status}")
http.clear()

x = soup.find("div", class_ = "related")
y = x.find("li", class_ = "nav-item nav-item-0")
z = y.find("a").text

versions = re.search(regex, z)
print(versions.group(0))

4.12.0


## <a name="ex2">Exercise 2: Extracting information from Wikipedia</a> [(&#8593;)](#content)

This exercise consists in extracting the birthdate of a list of actors from their Wikipedia page to infer their age. Consider for instance a list composed of Brad Pitt, Laurent Cantet, Jean-Paul Belmondo, Matthew McConaughey, Marion Cotillard, ...

To this aim, take a look at one such Wikipedia page, verify whether a birthdate is reported, and take a look at the `.html` source code of the page (from your browser) to see where this information is located. 

First write a function to automatically retrieve the birthdate of each actor in the list. In a second step, convert this information into a "numerical date" (see codes below) and compute the difference with the current date to estimate the actors' age.

> Hints: 
> - note that the birth date is associated whith the class `class="nowrap date-lien bday"` (check source code of the web page);
> - useful object: `bs4.BeautifulSoup`, with its `find` method, see the [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/);
> - you can create an `Actor` class to collect useful attributes (see [here](https://scipy-lectures.org/intro/language/oop.html?highlight=classes) and [there](https://docs.python.org/3/tutorial/classes.html) for more details on defining classes in Python).
> 
>```python
>class Actor:
>    def __init__(self, firstname, name):
>        self.name = name
>        self.firstname = firstname
>    ...
>
>```

> Codes: one possible way to translate words into a numerical date to compute an age is
>```python
># Parse data (replace month by number)
>month = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']
>month_number = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
>
>for i in range(0, 12):
>    web_date = web_date.replace(month[i], month_number[i])
>    
># Parse data and find the date to translate it into a numerical value
>born = datetime.strptime(web_date, '%m %Y')
>now = date.today()
>
># Compute the age
>age = now - born.date()
>age.days / 356
>
>result = now.year - born.date().year - ((now.month, now.day) < (born.date().month, born.date().day))
>```

Your answers(s)

In [8]:
# Classe Actor
class Actor:
    # Initialisation de l'objet Actor avec des paramètres optionnels
    def __init__(self, firstname, name, sex="F", birthdate=""):
        self.name = name
        self.firstname = firstname
        self.sex = sex
        self.URL = "https://fr.wikipedia.org/wiki/" + firstname + "_" + name
        self.birthdate = birthdate
        # Appel de la méthode request pour récupérer la date de naissance depuis Wikipedia
        self.request()

    # Méthode pour calculer l'âge de l'acteur en fonction de sa date de naissance
    def calculate_age(self):
        today = datetime.today()
        age = today.year - self.birthdate.year - ((today.month, today.day) < (self.birthdate.month, self.birthdate.day))
        return age

    # Méthode pour définir la date de naissance de l'acteur à partir d'une chaîne de caractères
    def set_birthdate(self, birthdate):
        self.birthdate = datetime.strptime(birthdate, '%Y-%m-%d')
        self.age = self.calculate_age()

    # Méthode pour afficher la description de l'acteur
    def description(self):
        month = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']
        acteur = "acteur" if self.sex == "H" else "actrice"
        e = "" if self.sex == "H" else "e"
        if self.birthdate != "":
            print(f"L'{acteur} {self.firstname} {self.name} est né{e} le {self.birthdate.day} {month[self.birthdate.month-1]} {self.birthdate.year} et a aujourd'hui {self.age} ans.")
        else:
            print(f"L'{acteur} {self.firstname} {self.name} n'a pas été trouvé{e} sur Wikipédia.")

    # Méthode pour effectuer la requête HTTP et extraire la date de naissance depuis la page Wikipedia
    def request(self):
        response = http.request("GET", self.URL)
        x = None
        soup = None
        if response.status == 200:
            soup = BeautifulSoup(response.data, 'html.parser')
            # Recherche de la balise 'time' avec la classe 'nowrap date-lien bday'
            x = soup.find("time", class_="nowrap date-lien bday")
            if x is None:
                # Si la balise n'est pas trouvée, recherche avec la classe 'nowrap date-lien'
                x = soup.find("time", class_="nowrap date-lien")
            # Appel de la méthode set_birthdate pour définir la date de naissance
        if x != None:
            self.set_birthdate(x['datetime'])


In [9]:
# Liste d'objets Actor représentant différents acteurs
Actors = [Actor("Brad", "Pitt", "H"),
          Actor("Laurent", "Cantet", "H"), 
          Actor("Jean-Paul", "Belmondo", "H"), 
          Actor("Matthew", "McConaughey", "H"), 
          Actor("Marion", "Cotillard"), 
          Actor("Benjamin", "Renoux", "H"), 
          Actor("Jean", "Dujardin", "H"), 
          Actor("Bradley", "Cooper", "H"), 
          Actor("Pierre", "Chainais", "H"),
          Actor("Tom", "Cruise", "H"),
          Actor("Céline", "Fasulo"),
          Actor("Jane", "Birkin"),
          Actor("Ian", "Somerhalder", "H")]

# Affichage de la description de chaque acteur dans la liste
for actor in Actors:
    actor.description()

L'acteur Brad Pitt est né le 18 décembre 1963 et a aujourd'hui 59 ans.
L'acteur Laurent Cantet est né le 11 avril 1961 et a aujourd'hui 62 ans.
L'acteur Jean-Paul Belmondo est né le 9 avril 1933 et a aujourd'hui 90 ans.
L'acteur Matthew McConaughey est né le 4 novembre 1969 et a aujourd'hui 53 ans.
L'actrice Marion Cotillard est née le 30 septembre 1975 et a aujourd'hui 48 ans.
L'acteur Benjamin Renoux n'a pas été trouvé sur Wikipédia.
L'acteur Jean Dujardin est né le 19 juin 1972 et a aujourd'hui 51 ans.
L'acteur Bradley Cooper est né le 5 janvier 1975 et a aujourd'hui 48 ans.
L'acteur Pierre Chainais n'a pas été trouvé sur Wikipédia.
L'acteur Tom Cruise est né le 3 juillet 1962 et a aujourd'hui 61 ans.
L'actrice Céline Fasulo n'a pas été trouvée sur Wikipédia.
L'actrice Jane Birkin est née le 14 décembre 1946 et a aujourd'hui 76 ans.
L'acteur Ian Somerhalder est né le 8 décembre 1978 et a aujourd'hui 44 ans.
